In [1]:
import pandas as pd
import numpy as np
# 파일 경로를 지정하여 데이터프레임으로 읽기
df = pd.read_csv("C:\\Users\\Playdata\\Documents\\my\\SKN06\\2nd_PJ\\app_train.csv")
for idx, col in enumerate(df.columns):
    print(f"Index: {idx}, Column Name: {col}")

Index: 0, Column Name: SK_ID_CURR
Index: 1, Column Name: TARGET
Index: 2, Column Name: NAME_CONTRACT_TYPE
Index: 3, Column Name: CODE_GENDER
Index: 4, Column Name: FLAG_OWN_CAR
Index: 5, Column Name: FLAG_OWN_REALTY
Index: 6, Column Name: CNT_CHILDREN
Index: 7, Column Name: AMT_INCOME_TOTAL
Index: 8, Column Name: AMT_CREDIT
Index: 9, Column Name: AMT_ANNUITY
Index: 10, Column Name: AMT_GOODS_PRICE
Index: 11, Column Name: NAME_TYPE_SUITE
Index: 12, Column Name: NAME_INCOME_TYPE
Index: 13, Column Name: NAME_EDUCATION_TYPE
Index: 14, Column Name: NAME_FAMILY_STATUS
Index: 15, Column Name: NAME_HOUSING_TYPE
Index: 16, Column Name: REGION_POPULATION_RELATIVE
Index: 17, Column Name: DAYS_BIRTH
Index: 18, Column Name: DAYS_EMPLOYED
Index: 19, Column Name: DAYS_REGISTRATION
Index: 20, Column Name: DAYS_ID_PUBLISH
Index: 21, Column Name: OWN_CAR_AGE
Index: 22, Column Name: FLAG_MOBIL
Index: 23, Column Name: FLAG_EMP_PHONE
Index: 24, Column Name: FLAG_WORK_PHONE
Index: 25, Column Name: FLAG_CONT

In [2]:
columns_to_check = df.columns[96:116]  


df['FLAG_DOCUMENT'] = df[columns_to_check].apply(lambda row: 1 if (row == 1).any() else 0, axis=1)

# "FLAG_DOCUMENT_2" ~ "FLAG_DOCUMENT_21" 삭제
df = df.drop(columns = columns_to_check)
df = df.drop(columns = ["COMMONAREA_MEDI", "COMMONAREA_MODE", "NONLIVINGAPARTMENTS_MEDI", "NONLIVINGAPARTMENTS_MODE", "LIVINGAPARTMENTS_MEDI", "LIVINGAPARTMENTS_MODE",
                        "FLOORSMIN_MEDI", "FLOORSMIN_MODE", "YEARS_BUILD_MEDI", "YEARS_BUILD_MODE", "LANDAREA_MEDI", "LANDAREA_MODE", "BASEMENTAREA_MEDI", "BASEMENTAREA_MODE",
                        "NONLIVINGAREA_MEDI", "NONLIVINGAREA_MODE", "ELEVATORS_MEDI", "ELEVATORS_MODE", "APARTMENTS_MEDI", "APARTMENTS_MODE", "ENTRANCES_MEDI", "ENTRANCES_MODE",
                        "LIVINGAREA_MEDI", "LIVINGAREA_MODE", "FLOORSMAX_MEDI", "FLOORSMAX_MODE", "YEARS_BEGINEXPLUATATION_MEDI", "YEARS_BEGINEXPLUATATION_MODE",
                       ])

# threshold = 0.4 * len(df)  # null 값이 40% 이상일 때
# df = df.dropna(thresh=len(df) - threshold, axis=1)

# # AMT_REQ_CREDIT_BUREAU_ 변수들 결측치 0으로 채기기
# df.iloc[:, 47:53] = df.iloc[:, 47:53].fillna(0)
# df.iloc[:, 41] = df.iloc[:, 41].fillna(0)
# df

In [ ]:
import numpy as np
from scipy.stats import anderson, shapiro
import warnings

# FutureWarning 방지 설정
warnings.simplefilter(action='ignore', category=FutureWarning)

# 수치형 변수와 범주형 변수 구분
numerical_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# 수치형 변수 결측치 처리
for col in numerical_cols:
    # 결측치가 있는 경우에만 진행
    if df[col].isnull().sum() > 0:
        # 샘플 수가 5000을 초과하면 Anderson-Darling 테스트 사용, 그렇지 않으면 Shapiro-Wilk 테스트 사용
        if len(df[col].dropna()) > 5000:
            result = anderson(df[col].dropna(), dist='norm')
            if result.statistic < result.critical_values[2]:
                mean_value = df[col].mean()
                df[col] = df[col].fillna(mean_value).copy()
                print(f"Column '{col}'의 결측치를 평균값으로 대체하였습니다 (Anderson-Darling 테스트 사용).")
            else:
                median_value = df[col].median()
                df[col] = df[col].fillna(median_value).copy()
                print(f"Column '{col}'의 결측치를 중앙값으로 대체하였습니다 (Anderson-Darling 테스트 사용).")
        else:
            # 정규성 테스트 (Shapiro-Wilk Test 사용)
            stat, p = shapiro(df[col].dropna())
            # p-value가 0.05보다 크다면 데이터가 정규분포를 따를 가능성이 높다고 판단하여 평균값으로 대체
            if p > 0.05:
                mean_value = df[col].mean()
                df[col] = df[col].fillna(mean_value).copy()
                print(f"Column '{col}'의 결측치를 평균값으로 대체하였습니다.")
            else:
                # 정규분포가 아니거나 이상치가 많은 경우 중앙값으로 대체
                median_value = df[col].median()
                df[col] = df[col].fillna(median_value).copy()
                print(f"Column '{col}'의 결측치를 중앙값으로 대체하였습니다.")

# 범주형 변수 결측치 처리
for col in categorical_cols:
    # 결측치가 있는 경우 최빈값으로 대체
    if df[col].isnull().sum() > 0:
        mode_value = df[col].mode()[0]
        df[col] = df[col].fillna(mode_value).copy()
        print(f"Column '{col}'의 결측치를 최빈값으로 대체하였습니다.")

In [ ]:
import pandas as pd

def find_outliers_and_replace(df, whis=1.5):
    """
    주어진 데이터프레임에서 숫자형 열들에 대해 IQR 방식을 통해 이상치를 찾아
    이상치 값을 최소 또는 최대 허용 범위 값으로 대체하는 함수
    
    Parameters:
    - df: 데이터프레임
    - whis: IQR의 배수로, 기본값은 1.5
    
    Returns:
    - 이상치가 대체된 데이터프레임
    """
    # 숫자형 열만 선택
    numeric_columns = df.select_dtypes(include='number').columns
    
    for column_name in numeric_columns:
        # 1사분위수(Q1)와 3사분위수(Q3) 계산
        q1, q3 = df[column_name].quantile([0.25, 0.75])
        iqr = q3 - q1

        # 하한과 상한 계산
        lower_bound = q1 - iqr * whis
        upper_bound = q3 + iqr * whis

        # 이상치를 하한과 상한으로 대체
        df[column_name] = df[column_name].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x))
    
    return df

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 원본 데이터프레임 컬럼 수
original_column_count = df.shape[1]

# 범주형 및 True/False 컬럼 선택
categorical_columns = df.select_dtypes(include=['object', 'category', 'bool']).columns

# 클래스가 2개인 컬럼에는 Label Encoding, 3개 이상인 컬럼에는 One-Hot Encoding
for col in categorical_columns:
    unique_classes = df[col].nunique()  # 해당 컬럼의 클래스 개수 확인
    if unique_classes == 2:
        # Label Encoding 적용
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    elif unique_classes > 2:
        # One-Hot Encoding 적용
        df = pd.get_dummies(df, columns=[col], prefix=col)


df.head(10)